# Get a list of Targets that are part of a particular pathway
#### Create a POST request to the RaMP API

In [16]:
import requests
import json
import pandas as pd

ramp_url = "https://rampdb.nih.gov/api/analytes-from-pathways"

post_body = {
    "analyte_type": "gene",
    "names_or_ids": "names",
    "max_pathway_size": 500,
    "pathway": ["Disturbed pathways in Duchenne Muscular Dystrophy"]
}

#### do the POST request

In [17]:
response = requests.post(ramp_url, data=post_body)
response_json = json.loads(response.text)["data"]

#### Parse the result

In [18]:
target_uniprots = [id.split(":")[1] 
                   for entry in response_json 
                   for id in entry["sourceAnalyteIDs"].split(";") if id.strip().startswith("uniprot:")]

target_uniprots = list(set(target_uniprots))

print(pd.DataFrame(target_uniprots))

              0
0        E9PR09
1        P01375
2        B4DSV7
3    A0A5H1ZRQ8
4        G3V3K1
..          ...
220      P08505
221      Q4G0X0
222  A0A499FJK2
223  A0A494C1Q6
224      Q6TME4

[225 rows x 1 columns]


# Get gene-disease associations and drug activities for those targets
#### Create a POST request to the Pharos API

In [19]:
pharos_url = "https://pharos-api.ncats.io/graphql"

post_body = {
  "query": """query targetDetails($targets: [String]){
  targets(targets: $targets) {
    count
    targets(top: 100000) {
      name
      uniprot
      ligands(isdrug: true) {
        name
        synonyms {
          value
          name
        }
      }
      diseases {
        mondoID
        name
        mondoEquivalents {
          id
          name
        }
      }
    }
  }
}""",
  "variables": {"targets": target_uniprots},
  "operationName": "targetDetails"
}

#### do the POST request

In [20]:
response = requests.post(pharos_url, json=post_body)
response_json = json.loads(response.text)["data"]
target_list = response_json["targets"]["targets"]

#### Parse the disease results

In [21]:
disease_objects = [disease for target in target_list for disease in target["diseases"]]
disease_names = list(set(disease["name"] for disease in disease_objects))

print(pd.DataFrame(disease_objects))

gard_aliases = [alias["id"] for disease in disease_objects if disease["mondoEquivalents"] is not None for alias in disease["mondoEquivalents"] if alias["id"].startswith("GARD:")]
gard_aliases = list(set(gard_aliases))

           mondoID                                               name  \
0    MONDO:0005302  attention deficit hyperactivity disorder, inat...   
1    MONDO:0009807                                       osteosarcoma   
2    MONDO:0005083                                          psoriasis   
3    MONDO:0005575                                  colorectal cancer   
4    MONDO:0007959                                    medulloblastoma   
..             ...                                                ...   
299  MONDO:0005136                                            malaria   
300  MONDO:0006932                                    pulmonary edema   
301  MONDO:0004979                                             asthma   
302  MONDO:0004975                                  Alzheimer disease   
303  MONDO:0001566                              hypercalcemia disease   

                                      mondoEquivalents  
0    [{'id': 'COHD:438409', 'name': None}, {'id': '...  
1    [{'i

#### Parse the drug results

In [22]:
drug_objects = [drug for target in target_list for drug in target["ligands"]]
drug_names = list(set(drug["name"] for drug in drug_objects))

print(pd.DataFrame(drug_objects))

unii_aliases = [synonym["value"] for drug in drug_objects for synonym in drug["synonyms"] if synonym["name"] == "unii"]
unii_aliases = list(set(unii_aliases))

                    name                                           synonyms
0            abemaciclib  [{'value': '60UAB198HK', 'name': 'unii'}, {'va...
1             gabapentin  [{'value': '6CW7F3G59X', 'name': 'unii'}, {'va...
2             pregabalin  [{'value': '55JG375S6M', 'name': 'unii'}, {'va...
3   gabapentin enacarbil  [{'value': '75OCL1SPBQ', 'name': 'unii'}, {'va...
4            mirogabalin  [{'value': '81689826', 'name': 'PubChem'}, {'v...
5              sunitinib  [{'value': 'V99T50803M', 'name': 'unii'}, {'va...
6          cepharanthine  [{'value': '7592YJ0J6T', 'name': 'unii'}, {'va...
7              verapamil  [{'value': 'CJ0O37KU29', 'name': 'unii'}, {'va...
8             nifedipine  [{'value': 'I9ZF7L6G2L', 'name': 'unii'}, {'va...
9              diltiazem  [{'value': 'EE92BBP03H', 'name': 'unii'}, {'va...
10          nitrendipine  [{'value': '9B627AW319', 'name': 'unii'}, {'va...
11            nimodipine  [{'value': '57WA9QZ5WH', 'name': 'unii'}, {'va...
12          

## Get Disease Details
#### Create POST request to the RDAS API

In [23]:
rdas_url = "https://rdas.ncats.nih.gov/api/diseases"

post_body = {
    "query": """query diseaseQuery($where: GARDWhere) {
  gards(where: $where) {
    GardId
    GardName
    MeSH
    Synonyms
    COUNT_ARTICLES
    COUNT_TRIALS
    ClassificationLevel
  }
}""", 

  "variables": {"where": {"GardId_IN": gard_aliases}},
  "operationName": "diseaseQuery"
}

#### do the POST request

In [24]:
response = requests.post(rdas_url, json=post_body)
response_json = json.loads(response.text)["data"]

#### parse the results

In [25]:
gard_data = [{
    "GardId": obj["GardId"],
    "GardName": obj["GardName"],
    "COUNT_ARTICLES": obj["COUNT_ARTICLES"],
    "COUNT_TRIALS": obj["COUNT_TRIALS"],
    "ClassificationLevel": obj["ClassificationLevel"]
} for obj in response_json["gards"]]
df = pd.DataFrame(gard_data)
print(df)

          GardId                        GardName COUNT_ARTICLES COUNT_TRIALS  \
0   GARD:0007893                  Wilson disease           6487           48   
1   GARD:0007471  Progressive supranuclear palsy           5100          111   
2   GARD:0007459     Primary biliary cholangitis           9745          115   
3   GARD:0005784  Alpha-1-antitrypsin deficiency           3822           97   
4   GARD:0010480  Spinocerebellar ataxia type 29             11            1   
..           ...                             ...            ...          ...   
67  GARD:0010574       Åland Islands eye disease             13            0   
68  GARD:0006729  Hypokalemic periodic paralysis            768            4   
69  GARD:0005694            Retinitis pigmentosa           9853          196   
70  GARD:0004702      Embryonal rhabdomyosarcoma           1555            5   
71  GARD:0013215            Renal cell carcinoma          48773          674   

    ClassificationLevel  
0            

## Get Drug Details
#### Create GET request strings

In [26]:
drug_requests = [f"https://drugs.ncats.io/api/v1/substances({unii})?view=full" for unii in unii_aliases]

drug_objects = []
for req in drug_requests:
    
    drug_json = json.loads(requests.get(req).text)
    
    drug_obj = {
        "name": drug_json["_name"], 
        "approval_ID": drug_json["approvalID"],
        "substanceClass": drug_json["substanceClass"],
        "status": drug_json["status"]
    }
    if "structure" in drug_json:
        drug_obj["formula"] = drug_json["structure"]["formula"]
        drug_obj["mwt"] = drug_json["structure"]["mwt"]
    
    drug_objects.append(drug_obj)

df = pd.DataFrame(drug_objects)
print(df)

                             name approval_ID substanceClass    status  \
0   TACROLIMUS ANHYDROUS 8-EPIMER  294O3172M2       chemical  approved   
1                    PIMECROLIMUS  7KYV510875       chemical  approved   
2                  MEFENAMIC ACID  367589PJ2C       chemical  approved   
3                      GABAPENTIN  6CW7F3G59X       chemical  approved   
4                       DILTIAZEM  EE92BBP03H       chemical  approved   
5                      ISRADIPINE  YO1UK1S598       chemical  approved   
6                      EVEROLIMUS  9HW64Q8G6G       chemical  approved   
7        2-AMINO-4-METHYLPYRIDINE  394N1Z644H       chemical  approved   
8                       SUNITINIB  V99T50803M       chemical  approved   
9                    PREDNISOLONE  9PHQ9Y1OLM       chemical  approved   
10                     SIVELESTAT  DWI62G0P59       chemical  approved   
11                     PAROXETINE  41VRH5220H       chemical  approved   
12                   NITRENDIPINE  9B6